In [29]:
import tifffile as tiff
from PIL import Image
import os
import tifffile
import numpy as np

file = tifffile.imread(r"C:\rkka_Projects\cell_death_v1\Data\pathway\qpi\raw\necrosis\qpi_230721.132040.Necrosis_NaOH.001.Group1.A1.T001P01.tif")
file.shape

(36, 1059, 1059)

In [31]:
import napari
viewer = napari.Viewer()
viewer.add_image(file)

<Image layer 'file' at 0x26b0b011370>

In [28]:
import tifffile as tiff
from PIL import Image
import os
import tifffile
import numpy as np
from TCFile import TCFile

path = r"C:\rkka_Projects\cell_death_v1\Data\before_preprocess\raw\pathway\raw\Apoptosis\compressed_230512.160429.CD95_TNF_Ctr.003.CD95.A1.T001P15.TCF"
images = TCFile(path, '3D')
# file = tifffile.imread(path)

# Output directory for .jpg files
output_dir = r"C:\rkka_Projects\cell_death_v1\test"
os.makedirs(output_dir, exist_ok=True)

# Load the 3D .tif file
# images = tifffile.imread(path)
qpi_images = [np.sum(img, axis=0) for img in images]
qpi_max, qpi_min = np.max(qpi_images), np.min(qpi_images)

# Save each z-slice as a .jpg
for i, slice in enumerate(qpi_images):
    # Normalize slice to 0-255 range and convert to uint8
    normalized_slice = (slice - qpi_min) / (qpi_max - qpi_min) * 255
    normalized_slice = normalized_slice.astype(np.uint8)
    
    # Create output path
    output_path = os.path.join(output_dir, f'{i:05d}.jpg')
    
    # Convert NumPy array to image and save
    image = Image.fromarray(normalized_slice)
    image.save(output_path)

print("All slices saved successfully!")


All slices saved successfully!


In [1]:
import tifffile as tiff
from PIL import Image
import os
import tifffile
import dask.array as da
from TCFile import TCFile

path = r"C:\rkka_Projects\cell_death_v1\Data\before_preprocess\raw\pathway\raw\Apoptosis\compressed_230512.160429.CD95_TNF_Ctr.003.CD95.A1.T001P15.TCF"
images = TCFile(path, '3D')
# file = tifffile.imread(path)

# Output directory for .jpg files
output_dir = r"C:\rkka_Projects\cell_death_v1\test"
os.makedirs(output_dir, exist_ok=True)

# Load the 3D .tif file
# images = tifffile.imread(path)
qpi_3d_image = images.asdask() # t,z,y,x
qpi_images = da.sum(qpi_3d_image,axis=1)
qpi_max, qpi_min = da.max(qpi_images), da.min(qpi_images)

# Save each z-slice as a .jpg
normalized_qpi_images = (qpi_images - qpi_min) / (qpi_max - qpi_min) * 255
normalized_qpi_images = normalized_qpi_images.astype('<u1').compute()
for i in range(qpi_images.shape[0]):
    normalized_slice = normalized_qpi_images[i,:,:]    
    # Create output path
    output_path = os.path.join(output_dir, f'{i:05d}.jpg')
    
    # Convert NumPy array to image and save
    image = Image.fromarray(normalized_slice)
    image.save(output_path)

print("All slices saved successfully!")


All slices saved successfully!


In [247]:
import os
# if using Apple MPS, fall back to CPU for unsupported ops
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"
import numpy as np
import torch
import matplotlib.pyplot as plt
from PIL import Image

# select the device for computation
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")
print(f"using device: {device}")

if device.type == "cuda":
    # use bfloat16 for the entire notebook
    torch.autocast("cuda", dtype=torch.bfloat16).__enter__()
    # turn on tfloat32 for Ampere GPUs (https://pytorch.org/docs/stable/notes/cuda.html#tensorfloat-32-tf32-on-ampere-devices)
    if torch.cuda.get_device_properties(0).major >= 8:
        torch.backends.cuda.matmul.allow_tf32 = True
        torch.backends.cudnn.allow_tf32 = True
elif device.type == "mps":
    print(
        "\nSupport for MPS devices is preliminary. SAM 2 is trained with CUDA and might "
        "give numerically different outputs and sometimes degraded performance on MPS. "
        "See e.g. https://github.com/pytorch/pytorch/issues/84936 for a discussion."
    )

using device: cuda


In [248]:
import sys
sys.path.append(r"C:\rkka_Projects\sam2")

import sam2
from sam2.build_sam import build_sam2_video_predictor

sam2_checkpoint = r"C:\rkka_Projects\sam2\checkpoints\sam2.1_hiera_large.pt"
model_cfg = r"C:\rkka_Projects\sam2\sam2\configs\sam2.1\sam2.1_hiera_l.yaml"

predictor = build_sam2_video_predictor(model_cfg, sam2_checkpoint, device=device)

In [249]:
video_dir = r"C:\rkka_Projects\cell_death_v1\test"
inference_state = predictor.init_state(video_path=video_dir)
predictor.reset_state(inference_state=inference_state)

frame loading (JPEG):   0%|          | 0/12 [00:00<?, ?it/s]

frame loading (JPEG): 100%|██████████| 12/12 [00:00<00:00, 30.43it/s]


In [250]:
label_image = tifffile.imread('Labels.tif')
masks = {}
for idx, label in enumerate(list(np.unique(label_image))):
    mask = np.zeros(label_image.shape)
    mask[label_image==label] = label
    if label==0:
        pass
    else:
        masks[idx] = mask, label

In [251]:
frame_idx = 0
for i in range(len(masks)):
    obj_id = i+1
    mask = masks[obj_id][0]
    frame_idx, obj_ids, video_res_masks = predictor.add_new_mask(
        inference_state=inference_state,
        frame_idx=frame_idx,
        obj_id=obj_id,
        mask=mask
    )

C:\rkka_Projects\sam2\sam2\sam2_video_predictor.py:961: UserWarning: cannot import name '_C' from 'sam2' (C:\rkka_Projects\sam2\sam2\__init__.py)

Skipping the post-processing step due to the error above. You can still use SAM 2 and it's OK to ignore the error above, although some post-processing functionality may be limited (which doesn't affect the results in most cases; see https://github.com/facebookresearch/sam2/blob/main/INSTALL.md).
  pred_masks_gpu = fill_holes_in_mask_scores(


In [252]:
# run propagation throughout the video and collect the results in a dict
video_segments = {}  # video_segments contains the per-frame segmentation results

# run propagation throughout the video and collect the results in a dict
for out_frame_idx, out_obj_ids, out_mask_logits in predictor.propagate_in_video(inference_state, reverse=False):
    video_segments[out_frame_idx] = {
        out_obj_id: (out_mask_logits[i] > 0.0).cpu().numpy()
        for i, out_obj_id in enumerate(out_obj_ids)
    }


propagate in video: 100%|██████████| 12/12 [02:36<00:00, 13.07s/it]


In [253]:
seg = np.zeros((12, 1059, 1059))

for i in range(len(video_segments)):
    temp = np.zeros((1059, 1059))
    for j in range(len(masks)):
        idx = j+1
        temp += idx * video_segments[i][idx][0].astype(int)

    seg[i] = temp

In [ ]:
import napari
viewer = napari.Viewer()

In [243]:
viewer.add_image(file)

<Image layer 'file' at 0x1d957cf2e70>

In [244]:
viewer.add_image(seg)

<Image layer 'seg' at 0x1d95675af90>

In [137]:
seg[seg==9] = 4

In [219]:
from skimage import measure
props = measure.regionprops(seg.astype(int)[0])
props_table = measure.regionprops_table(seg.astype(int), properties=['area', 'bbox', 'centroid'])

In [220]:
import pandas as pd
props_table = measure.regionprops_table(seg.astype(int)[0], properties=['area', 'bbox', 'centroid'])
pd.DataFrame(props_table)

,area,bbox-0,bbox-1,bbox-2,bbox-3,centroid-0,centroid-1
0,23701.0,0,17,210,190,99.393865,107.104510
1,17835.0,79,196,270,326,172.379479,260.883824
2,14691.0,0,297,164,439,66.458648,381.800422
3,25503.0,29,461,251,621,133.094695,537.370309
4,12437.0,184,665,290,834,240.714320,742.218381
5,22959.0,74,817,272,995,162.880831,897.783832
6,13194.0,312,481,410,682,354.514855,568.763226
7,22167.0,363,508,484,856,428.272703,696.885505
8,34995.0,469,343,773,518,607.958823,421.658894
9,12166.0,579,161,663,364,621.540605,254.052523


In [254]:
import numpy as np
import os
import imageio
from skimage import measure

# Define constants
file_num = 4
time_length = 12
labels = seg.astype(int)  # Assuming 'seg' is already defined
output_dir = r"C:\rkka_Projects\cell_death_v1\Data\pathway\qpi\output_pngs\necrosis"  # Directory to save PNGs
os.makedirs(output_dir, exist_ok=True)  # Ensure the directory exists

ri_max, ri_min = np.max(file), np.min(file)

for time in range(time_length):  # Iterate through time frames
    props = measure.regionprops(labels[time])  # Get properties for the current frame

    for i, prop in enumerate(props):  # Iterate through label IDs and their properties
        label_id = prop.label  # Get the label ID
        bbox = prop.bbox       # Get the bounding box (min_row, min_col, max_row, max_col)

        temp_file = file[time].copy()  # Copy the file array for the current time
        temp_label = labels[time].copy()  # Copy the label array for the current time

        # Zero-out non-matching labels in the temp_file
        temp_file[temp_label != label_id] = ri_min

        # Crop the file array using the bounding box
        min_row, min_col, max_row, max_col = bbox
        cropped_temp_file = temp_file[min_row:max_row, min_col:max_col]

        # Normalize the cropped array for saving as PNG (0 to 255)
        normalized_temp = ((cropped_temp_file - ri_min) / (ri_max - ri_min) * 255).astype(np.uint8)

        # Construct file name with time and label_id
        filename = f"time_{time:02d}_file_{file_num}_label_{label_id}.png"
        file_path = os.path.join(output_dir, filename)

        # Save the cropped PNG file
        imageio.imwrite(file_path, normalized_temp)
        print(f"Saved: {file_path}")

Saved: C:\rkka_Projects\cell_death_v1\Data\pathway\qpi\output_pngs\necrosis\time_00_file_4_label_1.png
Saved: C:\rkka_Projects\cell_death_v1\Data\pathway\qpi\output_pngs\necrosis\time_00_file_4_label_2.png
Saved: C:\rkka_Projects\cell_death_v1\Data\pathway\qpi\output_pngs\necrosis\time_00_file_4_label_3.png
Saved: C:\rkka_Projects\cell_death_v1\Data\pathway\qpi\output_pngs\necrosis\time_00_file_4_label_4.png
Saved: C:\rkka_Projects\cell_death_v1\Data\pathway\qpi\output_pngs\necrosis\time_00_file_4_label_5.png
Saved: C:\rkka_Projects\cell_death_v1\Data\pathway\qpi\output_pngs\necrosis\time_00_file_4_label_6.png
Saved: C:\rkka_Projects\cell_death_v1\Data\pathway\qpi\output_pngs\necrosis\time_00_file_4_label_7.png
Saved: C:\rkka_Projects\cell_death_v1\Data\pathway\qpi\output_pngs\necrosis\time_00_file_4_label_8.png
Saved: C:\rkka_Projects\cell_death_v1\Data\pathway\qpi\output_pngs\necrosis\time_00_file_4_label_9.png
Saved: C:\rkka_Projects\cell_death_v1\Data\pathway\qpi\output_pngs\necros